In [1]:
from pathlib import Path
import traceback
import pandas as pd
import numpy as np
import re

In [2]:
"""
OBJECTIVE:
Investigate if it's better to directly concatenate dataframes, or to merge dataframes first
and select out columns and convert them to a series with pd.concat
"""

"\nOBJECTIVE:\nInvestigate if it's better to directly concatenate dataframes, or to merge dataframes first\nand select out columns and convert them to a series with pd.concat\n"

In [3]:
df1_list = [{"Chrom": "NW233434", 
                "GenomicModBase": 231, 
                "Rep1_A_BS": 4, 
                "Rep1_C_BS": 1, 
                "Rep1_G_BS": 1, 
                "Rep1_T_BS": 1, 
                "Rep1_TotalCoverage_BS": 2}, 

                {"Chrom": "NW233438", 
                "GenomicModBase": 231, 
                "Rep1_A_BS": 4, 
                "Rep1_C_BS": 1, 
                "Rep1_G_BS": 1, 
                "Rep1_T_BS": 1, 
                "Rep1_TotalCoverage_BS": 5},

                {"Chrom": "NW233435", 
                 "GenomicModBase": 331, 
                 "Rep1_A_BS": None, 
                 "Rep1_C_BS": None, 
                 "Rep1_G_BS": None, 
                 "Rep1_T_BS": None, 
                 "Rep1_TotalCoverage_BS": 3},

                {"Chrom": "NW233433", 
                "GenomicModBase": 230, 
                "Rep1_A_BS": 3, 
                "Rep1_C_BS": 1, 
                "Rep1_G_BS": 1, 
                "Rep1_T_BS": 1, 
                "Rep1_TotalCoverage_BS": 0},
                
                {"Chrom": "NW233432", 
                "GenomicModBase": 229, 
                "Rep1_A_BS": 2, 
                "Rep1_C_BS": 6, 
                "Rep1_G_BS": 4, 
                "Rep1_T_BS": 7, 
                "Rep1_TotalCoverage_BS": 0}]
df1 = pd.DataFrame(df1_list).drop_duplicates()

In [4]:
df2_list = [{"Chrom": "NW233434", 
            "GenomicModBase": 231, 
            "Rep2_A_BS": 4, 
            "Rep2_C_BS": 1, 
            "Rep2_G_BS": 1, 
            "Rep2_T_BS": 1, 
            "Rep2_TotalCoverage_BS": 77}, 

            {"Chrom": "NW233438", 
             "GenomicModBase": 231, 
             "Rep2_A_BS": 4, 
             "Rep2_C_BS": 1, 
             "Rep2_G_BS": 1, 
             "Rep2_T_BS": 1, 
             "Rep2_TotalCoverage_BS": 88},

            {"Chrom": "NW233435", 
             "GenomicModBase": 331, 
             "Rep2_A_BS": None, 
             "Rep2_C_BS": None, 
             "Rep2_G_BS": None, 
             "Rep2_T_BS": None, 
             "Rep2_TotalCoverage_BS": 99},

            {"Chrom": "NW233433", 
             "GenomicModBase": 230, 
             "Rep2_A_BS": 3, 
             "Rep2_C_BS": 1, 
             "Rep2_G_BS": 1, 
             "Rep2_T_BS": 1, 
             "Rep2_TotalCoverage_BS": 100},
            
            {"Chrom": "NW233432", 
             "GenomicModBase": 229, 
             "Rep2_A_BS": 2, 
             "Rep2_C_BS": 6, 
             "Rep2_G_BS": 4, 
             "Rep2_T_BS": 7, 
             "Rep2_TotalCoverage_BS": 40}]
df2 = pd.DataFrame(df2_list).drop_duplicates()

In [11]:
%%timeit
df_list = [df1, df2]

def manual_concat(df_list):
    new_list = []
    for df in df_list:
        cov_col = [col for col in df.columns if re.search("_TotalCoverage_", col)]
        if cov_col:
            diff_cols = (df.columns.difference(cov_col, sort = False))
            df = df.drop(columns = diff_cols).rename(columns = {cov_col[0]: "TotalCoverage"})
            new_list.append(df)
    
    df_concat = pd.concat(new_list, ignore_index = True)
    return df_concat

manual_concat(df_list)

840 μs ± 287 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [10]:
%%timeit

def merge_first(df1, df2):
    selected_cols = (df1.columns.tolist())[0:2]
    df_merged = pd.merge(df1, df2,
                         on = selected_cols,
                         how = "outer",
                         sort = False)
    cov_col = [col for col in df_merged.columns if re.search("_TotalCoverage_", col)]
    series_list = [df_merged[col] for col in cov_col]
    single_series = pd.concat(series_list, ignore_index = True)
    return single_series

merge_first(df1, df2)

1.58 ms ± 461 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
